## Preprocessing

In [1]:
 # Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Tensorflow
import tensorflow as tf

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cols_to_drop = ["EIN", "NAME"]
df.drop(columns=cols_to_drop, inplace=True)
df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']

for col in cols:
    print("Column Name:", col)
    
    unique_values_count = df[col].nunique()
    print("Number of unique values:", unique_values_count)
    
    print("Value counts:")
    print(df[col].value_counts())
    
    print("-----------------------------")

Column Name: APPLICATION_TYPE
Number of unique values: 17
Value counts:
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64
-----------------------------
Column Name: AFFILIATION
Number of unique values: 6
Value counts:
AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64
-----------------------------
Column Name: CLASSIFICATION
Number of unique values: 71
Value counts:
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64
-----------------------------
Column Name: USE_CASE
Numb

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
min_count = 600

# Search for number of entries eacha pplicated appears.
app_type_counts = df['APPLICATION_TYPE'].value_counts()

# Find the application types that are below the count threshold
app_types_to_change = app_type_counts[app_type_counts < min_count].index

# Go through the dataframe and change those types to "Other"
for app_type in app_types_to_change:
    df.loc[df['APPLICATION_TYPE'] == app_type, 'APPLICATION_TYPE'] = 'Other'

# Print out the updated counts to check the replacements
print(df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64


In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
number_of_classes = df["CLASSIFICATION"].value_counts()

# Find classifications that are less than 500
classes_below_threshold = number_of_classes[number_of_classes < 400]
classes_to_change = classes_below_threshold.index.tolist()

# Go through and change each one to "Other"
for class_name in classes_to_change:
    df.loc[df['CLASSIFICATION'] == class_name, 'CLASSIFICATION'] = 'Other'

# Print out counts to make sure the changes happened
print(df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_df = pd.get_dummies(df, dtype=int)
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = encoded_df.drop(columns=["IS_SUCCESSFUL"]).values
y = encoded_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)
print(X_train.shape, X_test.shape)

(25724, 43) (8575, 43)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

trained_scaler = scaler.fit(X_train)

# Use the scaler to transform both training and test data
X_train_scaled = trained_scaler.transform(X_train)
X_test_scaled = trained_scaler.transform(X_test)

# Check to make sure scaling happened correctly
print("Training data scaled:", X_train_scaled)
print("Test data scaled:", X_test_scaled)

Training data scaled: [[ 0.01079982 -0.01334191 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 ...
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.02964949 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]]
Test data scaled: [[ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 ...
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.02925687
  -0.02925687]
 [ 0.01079982 -0.03089076 -0.15519457 ... -0.07397412  0.0

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

cnn = tf.keras.models.Sequential()

# First hidden layer
cnn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))


# Add an initial Dense layer with input dimensions and ReLU activation
cnn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer with a sigmoid activation function for probabilities
cnn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Display the model architecture
cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                      │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
cnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = cnn.fit(X_train, y_train, epochs=20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 437us/step - accuracy: 0.4988 - loss: 32243.8516
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - accuracy: 0.4917 - loss: 7218.2119
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.5104 - loss: 19341.7070
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.5174 - loss: 66941.4922
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.5187 - loss: 30922.2012
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - accuracy: 0.5015 - loss: 12720.6543
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step - accuracy: 0.4872 - loss: 6106.9771
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.5288 - loss: 6842.0498
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step - accuracy: 0.4794 - loss: 29100.9531
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step - accuracy: 0.4925 - loss: 27971.7402
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.5103 - loss: 

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = cnn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 683us/step - accuracy: 0.6617 - loss: 0.6485
Loss: 0.6485320925712585, Accuracy: 0.6616909503936768


In [19]:
# Export our model to HDF5 file
cnn.save("CharityCharity.h5")